# Tutorial 5: SQL Basics
SQL is a standard language for storing, handling and retrieving data from databases. Relatively easy to learn with intuitive syntax, very commonly used in databases around the world. \
Now a database is created locally in your computer by loading a Pandas DataFrame with the following code:

In [1]:
import sqlite3
import pandas as pd

#download nba data in csv format from online, save it as a DataFrame
download_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"
nba = pd.read_csv(download_url)

#convert date_game column string into date object, originally in month/day/Year format
nba.date_game = pd.to_datetime(nba.date_game, format='%m/%d/%Y', errors='coerce').dt.date

identifier_var_list = ['gameorder', 'game_id', '_iscopy', 'year_id', 'date_game', 'team_id', 'fran_id', 'pts', 'opp_id', 'opp_fran', 'opp_pts']

#elo rating represents team performance, changes by every match based on final score and whether the match takes place at home. 
#Higher elo rating means better overall performance.
#elo_i represents elo rating for main team before the match. (opp_elo_i for the opponent team)
#elo_n represents elo rating for main team after the match. (opp_elo_n for the opponent team)

match_details_list = ['gameorder', 'game_id', '_iscopy', 'year_id', 'date_game', 'seasongame', 'team_id', 'fran_id', 'pts', 'elo_i',
                      'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i', 'opp_elo_n', 'game_location', 
                      'game_result', 'forecast']

In [2]:
#Create a local database in the own computer, not connected to real online databases
conn = sqlite3.connect("nba.db")

In [3]:
#Store the specific columns of nba DataFrame into the locally created database. The number of rows are printed at the end of this operation.
nba[identifier_var_list].to_sql('match_records', conn, if_exists='replace', index=False)
nba[match_details_list].to_sql('match_records_with_forecast', conn, if_exists='replace', index=False)

126314

In [4]:
conn.close()   # Don't forget to close the connection!

### 5.1 SQL retrieving data

In [5]:
#create a database cursor for fetching data
conn = sqlite3.connect("nba.db")
c = conn.cursor()

#state one or more column names inside "SELECT column_names FROM table_name" to fetch the data in a table
query = """
SELECT game_id, _iscopy, date_game, team_id, fran_id, opp_id, opp_fran, pts, opp_pts
FROM match_records;
"""

core_data = c.execute(query)
for ii in range(5):
    print(core_data.fetchone())#each row is printed as tuple

('194611010TRH', 0, '1946-11-01', 'TRH', 'Huskies', 'NYK', 'Knicks', 66, 68)
('194611010TRH', 1, '1946-11-01', 'NYK', 'Knicks', 'TRH', 'Huskies', 68, 66)
('194611020CHS', 0, '1946-11-02', 'CHS', 'Stags', 'NYK', 'Knicks', 63, 47)
('194611020CHS', 1, '1946-11-02', 'NYK', 'Knicks', 'CHS', 'Stags', 47, 63)
('194611020DTF', 0, '1946-11-02', 'DTF', 'Falcons', 'WSC', 'Capitols', 33, 50)


- The following code retrieves data from all columns in match_records table.

In [6]:
query = """
SELECT * FROM match_records;
"""
core_data2 = c.execute(query)
for ii in range(5):
    print(core_data2.fetchone())#each row is printed as tuple

(1, '194611010TRH', 0, 1947, '1946-11-01', 'TRH', 'Huskies', 66, 'NYK', 'Knicks', 68)
(1, '194611010TRH', 1, 1947, '1946-11-01', 'NYK', 'Knicks', 68, 'TRH', 'Huskies', 66)
(2, '194611020CHS', 0, 1947, '1946-11-02', 'CHS', 'Stags', 63, 'NYK', 'Knicks', 47)
(2, '194611020CHS', 1, 1947, '1946-11-02', 'NYK', 'Knicks', 47, 'CHS', 'Stags', 63)
(3, '194611020DTF', 0, 1947, '1946-11-02', 'DTF', 'Falcons', 33, 'WSC', 'Capitols', 50)


In [7]:
#Sort the data with team_id column first and then with date_game column, both sorted in ascending order
query = """
SELECT date_game, team_id, fran_id, opp_id, opp_fran, pts, opp_pts FROM match_records ORDER BY team_id, date_game;
"""

match_data = c.execute(query)
match_data.fetchall()

[('1967-10-13', 'ANA', 'Stars', 'OAK', 'Squires', 129, 134),
 ('1967-10-15', 'ANA', 'Stars', 'DNR', 'Nuggets', 105, 110),
 ('1967-10-16', 'ANA', 'Stars', 'DLC', 'Spurs', 125, 129),
 ('1967-10-18', 'ANA', 'Stars', 'INA', 'Pacers', 103, 106),
 ('1967-10-24', 'ANA', 'Stars', 'KEN', 'Colonels', 127, 128),
 ('1967-10-26', 'ANA', 'Stars', 'MNM', 'Floridians', 115, 99),
 ('1967-10-29', 'ANA', 'Stars', 'OAK', 'Squires', 120, 123),
 ('1967-10-31', 'ANA', 'Stars', 'OAK', 'Squires', 126, 123),
 ('1967-11-03', 'ANA', 'Stars', 'PTP', 'Condors', 101, 94),
 ('1967-11-04', 'ANA', 'Stars', 'KEN', 'Colonels', 103, 104),
 ('1967-11-05', 'ANA', 'Stars', 'MNM', 'Floridians', 92, 119),
 ('1967-11-06', 'ANA', 'Stars', 'DNR', 'Nuggets', 100, 121),
 ('1967-11-08', 'ANA', 'Stars', 'HSM', 'Spirits', 104, 100),
 ('1967-11-13', 'ANA', 'Stars', 'NOB', 'Sounds', 98, 106),
 ('1967-11-14', 'ANA', 'Stars', 'HSM', 'Spirits', 106, 113),
 ('1967-11-15', 'ANA', 'Stars', 'DLC', 'Spurs', 116, 105),
 ('1967-11-19', 'ANA', 'St

In [8]:
#Sort the data with team_id column first in ascending order, then with date_game column in descending order.
query = """
SELECT date_game, team_id, fran_id, opp_id, opp_fran, pts, opp_pts FROM match_records ORDER BY team_id, date_game DESC;
"""

stock_returns = c.execute(query)
stock_returns.fetchall()

[('1968-03-24', 'ANA', 'Stars', 'OAK', 'Squires', 147, 135),
 ('1968-03-21', 'ANA', 'Stars', 'NOB', 'Sounds', 89, 105),
 ('1968-03-19', 'ANA', 'Stars', 'PTP', 'Condors', 122, 150),
 ('1968-03-18', 'ANA', 'Stars', 'PTP', 'Condors', 118, 135),
 ('1968-03-17', 'ANA', 'Stars', 'KEN', 'Colonels', 107, 115),
 ('1968-03-15', 'ANA', 'Stars', 'OAK', 'Squires', 116, 113),
 ('1968-03-10', 'ANA', 'Stars', 'DLC', 'Spurs', 110, 112),
 ('1968-03-07', 'ANA', 'Stars', 'MNM', 'Floridians', 115, 124),
 ('1968-03-05', 'ANA', 'Stars', 'PTP', 'Condors', 110, 123),
 ('1968-03-03', 'ANA', 'Stars', 'DLC', 'Spurs', 114, 126),
 ('1968-03-02', 'ANA', 'Stars', 'INA', 'Pacers', 119, 124),
 ('1968-03-01', 'ANA', 'Stars', 'NOB', 'Sounds', 114, 118),
 ('1968-02-27', 'ANA', 'Stars', 'DLC', 'Spurs', 132, 110),
 ('1968-02-25', 'ANA', 'Stars', 'HSM', 'Spirits', 114, 109),
 ('1968-02-22', 'ANA', 'Stars', 'MNM', 'Floridians', 112, 128),
 ('1968-02-20', 'ANA', 'Stars', 'PTP', 'Condors', 114, 121),
 ('1968-02-17', 'ANA', 'Sta

### 5.2 Data Filtering
`WHERE` keyword is used for filtering the query, followed by a condition. \
In the following code, rows from `match_records` where `pts < opp_pts` and `pts >= 105` are fetched. \
All the fetched data represent matches in which the main team loses but scored 105 or above. 

In [9]:
query = """
SELECT date_game, team_id, fran_id, opp_id, opp_fran, pts, opp_pts 
FROM match_records WHERE pts < opp_pts AND pts >= 105
ORDER BY team_id, date_game;
"""

match_data1 = c.execute(query)
match_data1.fetchall()

[('1967-10-13', 'ANA', 'Stars', 'OAK', 'Squires', 129, 134),
 ('1967-10-15', 'ANA', 'Stars', 'DNR', 'Nuggets', 105, 110),
 ('1967-10-16', 'ANA', 'Stars', 'DLC', 'Spurs', 125, 129),
 ('1967-10-24', 'ANA', 'Stars', 'KEN', 'Colonels', 127, 128),
 ('1967-10-29', 'ANA', 'Stars', 'OAK', 'Squires', 120, 123),
 ('1967-11-14', 'ANA', 'Stars', 'HSM', 'Spirits', 106, 113),
 ('1967-12-02', 'ANA', 'Stars', 'INA', 'Pacers', 117, 132),
 ('1967-12-08', 'ANA', 'Stars', 'DLC', 'Spurs', 110, 123),
 ('1967-12-20', 'ANA', 'Stars', 'INA', 'Pacers', 142, 144),
 ('1968-01-02', 'ANA', 'Stars', 'OAK', 'Squires', 121, 141),
 ('1968-01-10', 'ANA', 'Stars', 'HSM', 'Spirits', 120, 122),
 ('1968-01-11', 'ANA', 'Stars', 'NOB', 'Sounds', 124, 131),
 ('1968-01-12', 'ANA', 'Stars', 'KEN', 'Colonels', 108, 117),
 ('1968-01-14', 'ANA', 'Stars', 'DLC', 'Spurs', 123, 130),
 ('1968-01-17', 'ANA', 'Stars', 'MNM', 'Floridians', 115, 119),
 ('1968-01-19', 'ANA', 'Stars', 'DNR', 'Nuggets', 110, 117),
 ('1968-01-28', 'ANA', 'Star

- Count the total number of matches that satisfy the criteria above (main team loses but scores 105 or above):

In [10]:
query = """
SELECT COUNT(pts)
FROM match_records WHERE pts < opp_pts AND pts >= 105;
"""

match_data_count = c.execute(query)
out = match_data_count.fetchone()
print(out[0])

18733


In [11]:
#Using fetchall() method:
match_data_count = c.execute(query)
out = match_data_count.fetchall()
print(out[0][0])

18733


In [12]:
#Using len() function with previous query:
query = """
SELECT date_game, team_id, fran_id, opp_id, opp_fran, pts, opp_pts 
FROM match_records WHERE pts < opp_pts AND pts >= 105
ORDER BY team_id, date_game;
"""

match_data = c.execute(query)
len(match_data.fetchall())

18733

- Evaluate the maximum score obtained by the main team:

In [13]:
query = """
SELECT MAX(pts)
FROM match_records;
"""

max_pts = c.execute(query)
out = max_pts.fetchall()
print(out[0][0])

186


- Evaluate the maximum score obtained by each team using `GROUP BY` keyword:

In [14]:
query = """
SELECT team_id, fran_id, MAX(pts)
FROM match_records GROUP BY team_id ORDER BY team_id;
"""
max_pts = c.execute(query)
max_pts.fetchall()

[('ANA', 'Stars', 147),
 ('AND', 'Packers', 123),
 ('ATL', 'Hawks', 155),
 ('BAL', 'Wizards', 156),
 ('BLB', 'Baltimore', 126),
 ('BOS', 'Celtics', 173),
 ('BRK', 'Nets', 134),
 ('BUF', 'Clippers', 145),
 ('CAP', 'Wizards', 126),
 ('CAR', 'Spirits', 156),
 ('CHA', 'Hornets', 133),
 ('CHH', 'Pelicans', 141),
 ('CHI', 'Bulls', 156),
 ('CHO', 'Hornets', 122),
 ('CHP', 'Wizards', 129),
 ('CHS', 'Stags', 115),
 ('CHZ', 'Wizards', 142),
 ('CIN', 'Kings', 165),
 ('CLE', 'Cavaliers', 154),
 ('CLR', 'Rebels', 92),
 ('DAL', 'Mavericks', 156),
 ('DEN', 'Nuggets', 184),
 ('DET', 'Pistons', 186),
 ('DLC', 'Spurs', 158),
 ('DNA', 'Nuggets', 156),
 ('DNN', 'Denver', 108),
 ('DNR', 'Nuggets', 153),
 ('DTF', 'Falcons', 92),
 ('FLO', 'Floridians', 155),
 ('FTW', 'Pistons', 127),
 ('GSW', 'Warriors', 162),
 ('HOU', 'Rockets', 156),
 ('HSM', 'Spirits', 149),
 ('INA', 'Pacers', 177),
 ('IND', 'Pacers', 150),
 ('INJ', 'Jets', 110),
 ('INO', 'Olympians', 112),
 ('KCK', 'Kings', 146),
 ('KCO', 'Kings', 140),


- Evaluate the maximum and minimum scores obtained by each team using GROUP BY keyword:

In [15]:
query = """
SELECT team_id, fran_id, MAX(pts), MIN(pts)
FROM match_records GROUP BY team_id ORDER BY team_id;
"""
max_min_pts = c.execute(query)
max_min_pts.fetchall()

[('ANA', 'Stars', 147, 85),
 ('AND', 'Packers', 123, 50),
 ('ATL', 'Hawks', 155, 58),
 ('BAL', 'Wizards', 156, 79),
 ('BLB', 'Baltimore', 126, 50),
 ('BOS', 'Celtics', 173, 43),
 ('BRK', 'Nets', 134, 72),
 ('BUF', 'Clippers', 145, 63),
 ('CAP', 'Wizards', 126, 71),
 ('CAR', 'Spirits', 156, 83),
 ('CHA', 'Hornets', 133, 59),
 ('CHH', 'Pelicans', 141, 64),
 ('CHI', 'Bulls', 156, 49),
 ('CHO', 'Hornets', 122, 66),
 ('CHP', 'Wizards', 129, 92),
 ('CHS', 'Stags', 115, 48),
 ('CHZ', 'Wizards', 142, 84),
 ('CIN', 'Kings', 165, 69),
 ('CLE', 'Cavaliers', 154, 57),
 ('CLR', 'Rebels', 92, 44),
 ('DAL', 'Mavericks', 156, 62),
 ('DEN', 'Nuggets', 184, 53),
 ('DET', 'Pistons', 186, 64),
 ('DLC', 'Spurs', 158, 79),
 ('DNA', 'Nuggets', 156, 80),
 ('DNN', 'Denver', 108, 51),
 ('DNR', 'Nuggets', 153, 0),
 ('DTF', 'Falcons', 92, 33),
 ('FLO', 'Floridians', 155, 84),
 ('FTW', 'Pistons', 127, 19),
 ('GSW', 'Warriors', 162, 64),
 ('HOU', 'Rockets', 156, 66),
 ('HSM', 'Spirits', 149, 80),
 ('INA', 'Pacers',

In [16]:
conn.close()   # Don't forget to close the connection!

### 5.3 Exercises

1. From `match_records_with_forecast` table in `nba.db` database, evaluate the average value of `forecast` column for each team using `team_id` identifer, \
considering matches taken place in and after year 2006, from `year_id` column. The higher the `forecast` value, the higher the chance of winning for the main team.

In [17]:
conn = sqlite3.connect("nba.db")
c = conn.cursor()
query = """
SELECT team_id, fran_id, AVG(forecast) FROM match_records_with_forecast WHERE year_id >= 2006 GROUP BY team_id;
"""
avg_forecast = c.execute(query)
avg_forecast.fetchall()

[('ATL', 'Hawks', 0.48919578209545483),
 ('BOS', 'Celtics', 0.5494751603344366),
 ('BRK', 'Nets', 0.4861292141955722),
 ('CHA', 'Hornets', 0.3833684342095888),
 ('CHI', 'Bulls', 0.54882850205889),
 ('CHO', 'Hornets', 0.4587750903658536),
 ('CLE', 'Cavaliers', 0.5140238086391061),
 ('DAL', 'Mavericks', 0.603520722255892),
 ('DEN', 'Nuggets', 0.5572520709195807),
 ('DET', 'Pistons', 0.4999589650407454),
 ('GSW', 'Warriors', 0.5021002435614033),
 ('HOU', 'Rockets', 0.5681585693713618),
 ('IND', 'Pacers', 0.5047672650405558),
 ('LAC', 'Clippers', 0.4891866658593026),
 ('LAL', 'Lakers', 0.5604871546688664),
 ('MEM', 'Grizzlies', 0.4910899672775839),
 ('MIA', 'Heat', 0.5747540851268819),
 ('MIL', 'Bucks', 0.42573205898305116),
 ('MIN', 'Timberwolves', 0.3687234890995029),
 ('NJN', 'Nets', 0.4039403965903615),
 ('NOH', 'Pelicans', 0.4915384819639281),
 ('NOK', 'Pelicans', 0.4252829896341463),
 ('NOP', 'Pelicans', 0.4376982013095239),
 ('NYK', 'Knicks', 0.4249278783563635),
 ('OKC', 'Thunder',

2. Count the number of matches in which the main team name (`fran_id`) ends with `"ers"`.

In [18]:
query = """
SELECT COUNT(fran_id) FROM match_records_with_forecast 
WHERE fran_id LIKE '%ers';
"""
ers = c.execute(query)
ers_count = ers.fetchone()
print(ers_count[0])

27797


3. For Golden States Warriors' 2014-15 season (`year_id : 2015, 'team_id : 'GSW'`), \
evaluate the number of matches that the team won and lost (from `game_result` column) respectively.

In [19]:
query = """
SELECT game_result, COUNT(game_result) FROM match_records_with_forecast 
WHERE year_id = 2015 AND team_id = 'GSW' 
GROUP BY game_result
"""
gsw_wins_2015 = c.execute(query)
gsw_wins_2015.fetchall()

[('L', 20), ('W', 83)]

In [20]:
conn.close()